In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import pandas as pd
import re
from datetime import datetime, timedelta

Payload = {"username": "secret", "password": "secret"}

In [0]:
set_top5 = pd.read_csv("Film-cast-top5.csv")
# add name_id to table
set_top5["name_id"] = set_top5['actorimdb'].str[25:34]

print(set_top5.shape)
set_top5.head(2)

(11751, 15)


,bomfilmtitle,filmimdb,year,notsample,filmrank,castrank,filmrole,actorname,actorimdb,birthyear,gender,race,bomlink,starpower-actor,name_id
0,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,4,1,Cast,Jennifer Garner,http://www.imdb.com/name/nm0004950/,1972.0,f,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0004950
1,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,5,2,Cast,Mark Ruffalo,http://www.imdb.com/name/nm0749263/,1967.0,m,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0749263


In [0]:
filled_date = pd.read_csv("Film-masterfile (from 'film-merge-8th.dta')-filled.csv")
# update releasedate with timestamp
filled_date['releasedate'] =  pd.to_datetime(filled_date['releasedate'])

# calculate the date for 10 weeks prior - releasedate
Weeks = 10
filled_date['prior10weeksDate'] = filled_date["releasedate"] - timedelta(days=7*Weeks)
filled_date_sub = filled_date[["bomtitle", "releasedate", "prior10weeksDate"]]
filled_date_sub.head(5)

,bomtitle,releasedate,prior10weeksDate
0,The Wizard of Oz (3D/IMAX),2013-09-20,2013-07-12
1,Singin' In the Rain (2012 re-release),2012-07-12,2012-05-03
2,Raiders of the Lost Ark (IMAX),2012-09-07,2012-06-29
3,Ghostbusters (30th Anniversary re-release),2014-08-29,2014-06-20
4,Back to the Future Day,2015-10-21,2015-08-12


In [0]:
# Merge filled_date table with part_filled_test
print(set_top5.shape)
full_columnSet = pd.merge(set_top5, filled_date_sub, how='left', left_on=["bomfilmtitle"], right_on = ["bomtitle"])
print(full_columnSet.shape)
# check how many np.nan values we have
print("Number of missing release date after merging: ", len(full_columnSet[full_columnSet['releasedate'].isna()]))
full_columnSet.head(5)

(11751, 16)
(11751, 19)
Number of missing release date after merging:  19


,bomfilmtitle,filmimdb,year,notsample,filmrank,castrank,filmrole,actorname,actorimdb,birthyear,gender,race,bomlink,starpower-actor,name_id,pageSource,bomtitle,releasedate,prior10weeksDate
0,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,4,1,Cast,Jennifer Garner,http://www.imdb.com/name/nm0004950/,1972.0,f,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0004950,NaN,13 Going on 30,2004-04-23,2004-02-13
1,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,5,2,Cast,Mark Ruffalo,http://www.imdb.com/name/nm0749263/,1967.0,m,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0749263,NaN,13 Going on 30,2004-04-23,2004-02-13
2,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,6,3,Cast,Judy Greer,http://www.imdb.com/name/nm0339460/,1975.0,f,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0339460,NaN,13 Going on 30,2004-04-23,2004-02-13
3,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,7,4,Cast,Andy Serkis,http://www.imdb.com/name/nm0785227/,1964.0,m,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0785227,NaN,13 Going on 30,2004-04-23,2004-02-13
4,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,8,5,Cast,Kathy Baker,http://www.imdb.com/name/nm0000834/,1950.0,f,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN,nm0000834,NaN,13 Going on 30,2004-04-23,2004-02-13


In [0]:
# Yield successive n-sized chunks from l. 
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
        
# modification for specific date extraction
def divide_chunks_date(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield [l[i:i+3], l[i+n-1]]

In [0]:
def searchForData(soup_, keywords_="var graphData", length_=8):
    sep = "-"
    dic_ = {}
    filter_ = soup_.findAll("script")
    for i in range(len(filter_)):
        if keywords_ in filter_[i].text:
            # we have fixed the location of graphData chunk
            dateList = list(divide_chunks_date(re.findall(r'\d+', filter_[i].text), length_))
            for j in range(len(dateList)):
                # turn the first list into timestamp
                key_ = pd.to_datetime(sep.join(dateList[j][0]), format='%Y-%m-%d')
                dic_[key_] = int(dateList[j][1])
            return dic_
    return dic_

In [0]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [0]:
# try to find the starPower value based on prior10weeksDate info
def findClosestPower(timestampDic_, releasedate_):
    if list(timestampDic_.keys()) == []:
        return 0
    if not bool(re.search(r'\d', str(releasedate_))):
        return 0
    stamp_key = nearest(list(timestampDic_.keys()), releasedate_)
    return timestampDic_[stamp_key]

In [0]:
import requests
import urllib
import urllib.request
import time
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import copy
import datetime 
from collections import Counter

import selenium
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def launch_selenium(names_list):
    from selenium.webdriver.common.by import By
    # mv chrome driver from Downloads to Applications 
    # chromedriver = "D:/chromedriver"
    chromedriver = "Desktop/Data/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver

    url = 'https://pro-labs.imdb.com/name/' + names_list + '/'
    opts = webdriver.ChromeOptions()
    opts.add_argument("user-agent=[user-agent string]")
    opts.add_argument("Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.117 Safari/537.36")
    driver = webdriver.Chrome(chromedriver, chrome_options=opts)
    driver.get(url)
    
    loginButton = driver.find_element_by_xpath('//a[@class="a-size- a-align- a-link- log_in"]')
    loginButton.click()
    
    time.sleep(.2)
    loginButton = driver.find_element_by_xpath(".//a[contains(@href,'https://pro.imdb.com/login/auth?')]")
    # loginButton = driver.find_element_by_xpath('//a[contains(@href,"https://pro.imdb.com/login/auth?")')
    # loginButton = driver.find_element(By.PARTIAL_LINK_TEXT, 'https://pro.imdb.com/login/auth?')
    loginButton.click()
    
    time.sleep(.2)
    username_form = driver.find_element_by_id("ap_email")
    username_form.send_keys(Payload["username"])

    password_form=driver.find_element_by_id('ap_password')
    password_form.send_keys(Payload["password"])
    password_form.send_keys(Keys.RETURN)

    return driver

In [0]:
# driver test
string_start = set_top5["name_id"][0]
set_top5["pageSource"] = np.nan
print(string_start)
# launch_selenium(string_start)

nm0004950


In [0]:
# run the function directly to get page source
def get_actor_rankings(driver, min_index, max_index, dataset_, keywordDate_ = "prior10weeksDate"):
    for i in range(min_index, max_index):
        name_id = dataset_["name_id"][i]
        # contains that star table
        url = 'https://pro.imdb.com/name/' + name_id + '?rf=cons_nm_meter&ref_=cons_nm_meter'
        driver.get(url)
        soup = BeautifulSoup(driver.page_source)
        # find all script records
        # soup = soup.findAll("script")
        temp_dic = searchForData(soup)
        PriorReleaseDate = dataset_[keywordDate_][i]
        starPower = findClosestPower(temp_dic, PriorReleaseDate)
        dataset_["starpower-actor"][i] = starPower
        if i%100 == 0:
            print(i)
    driver.quit()
    return dataset_

#### Check the result for star power values between 500 and 510:

In [0]:
# set_top5.shape[0]
set_top5_new_500_2nd = get_actor_rankings(launch_selenium(string_start), 500, 510, full_columnSet)